In [1]:
from SALib.sample import saltelli
from SALib.sample.fast_sampler import sample as fast_sampler
from SALib.sample.morris import sample as morris_sample
from SALib.sample import latin

from SALib.analyze import sobol
from SALib.analyze.fast import analyze as fast_analyzer
from SALib.analyze.morris import analyze as morris_analyze


from SALib.test_functions import Ishigami

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

sns.set_style('white')
sns.set_context('talk')

Because SALib concentrates on GSA methods, we implement our own local SA methods here.

In [3]:
def traditional_oat_sample(problem, N):
    """Create uniformly distributed samples for a 'traditional' or 'pure' OAT analysis.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs
    * N : int, total number of samples to generate
    
    Returns
    ==========
    * np.array of samples
    """
    bounds = problem.get('bounds')
    nominal = problem.get('nominal')
    num_factors = int(problem.get('num_vars'))
    
    samples_per_param = N / num_factors
    
    if samples_per_param % 1 != 0:
        raise ValueError("Require equal number of samples per parameter. \
        {} samples across {} params is {}".format(N, num_factors, samples_per_param))
    
    samples_per_param = int(samples_per_param)
    
    # experiment = np.empty((N+1, num_factors))
    experiment = np.array(nominal * (N+1)).reshape(N + 1, num_factors)
    
    cols = np.arange(num_factors)
    
    # Sample positions between bounds
    pos = 1
    for i in range(num_factors):
        
        min_val = bounds[i][0]
        max_val = bounds[i][1]
        
        p_samp = np.linspace(min_val, max_val, samples_per_param)
        
        subset = slice(pos, pos+samples_per_param)
        experiment[subset, i] = p_samp
        
        pos += samples_per_param
    # End for
    
    return experiment
# End traditional_oat_sample()


def oat_extremity_sample(problem):
    """Create extremity samples for a 'traditional' OAT analysis.
    
    Here we assume outputs will be affected at the extreme ends of the nominal bounds.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs        
    """
    bounds = problem.get('bounds')
    nominal = problem.get('nominal')
    num_vars = problem.get('num_vars')

    experiment = [nominal[:]]
    
    # Samples represent runs with values at the extremes of each parameter boundary
    for i in range(num_vars):
        min_val = bounds[i][0]
        max_val = bounds[i][1]
        
        after = i+1
        if i > 0:
            experiment.append(nominal[0:i] + [min_val] + nominal[after:])  # min
            experiment.append(nominal[0:i] + [max_val] + nominal[after:])  # max
        else:
            experiment.append([min_val] + nominal[after:])  # min
            experiment.append([max_val] + nominal[after:])  # max
            
    
    return np.array(experiment)
# End oat_extremity_sample()

    
def oat_analyze(problem, inputs, results):
    """Analyze a 'traditional' OAT result.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs
    * inputs : np.array, samples used to generate results. 
               The first row is expected to hold nominal values.
    * results : np.array, model run results obtained using the samples.
                The first row is expected to hold nominal results.
                
    Returns
    ==========
    * np.array
    """
    X = inputs
    Y = results
    
    num_results, num_factors = X.shape
    nominal_values = problem.get('nominal')
    grp_len = int((len(Y) - 1) / num_factors)

    # Set up result array (ignores one row as it holds the nominal results)
    # si_res = np.full((grp_len, num_factors), fill_value=np.nan)
    si_res = np.empty((grp_len, num_factors))

    col = 0
    nominal_res = Y[0]
    for i in range(1, num_results, grp_len):
    
        input_subset = X[i:i+grp_len, col]
        res_subset = Y[i:i+grp_len]

        diff = (input_subset - X[0, col])

        calc = ((res_subset - nominal_res) / diff) * (input_subset / res_subset)
        
        # si_res[col] = np.sum(calc) / grp_len
        si_res[:, col] = ((res_subset - nominal_res) / diff) * (input_subset / res_subset)

        col += 1
    # End for
    
    return si_res
# End oat_analyze()

Common variables and problem definitions

In [4]:
SEED_VALUE = 101
NUM_SALTELLI_SAMPLES = 500
NUM_FAST_SAMPLES = 5000
NUM_MORRIS_SAMPLES = 1000
NUM_OAT_SAMPLES = 5000

SPEC_WO_INACTIVE = {
  'num_vars': 3,
  'names': ['x1', 'x2', 'x3'],
  'bounds': [[-np.pi, np.pi]]*3
}

SPEC_WITH_INACTIVE = {
  'num_vars': 4,
  'names': ['x1', 'x2', 'x3', 'x4'],
  'bounds': [[-np.pi, np.pi]]*4
}

# problem specification for traditional OAT analysis
OAT_SPEC = {
  'num_vars': 4,
  'names': ['x1', 'x2', 'x3', 'x4'],
  'bounds': [[-np.pi, np.pi]]*4,
  'nominal': [1.0,] * 4
}

In [ ]:
### Define SALib problem spec
CIM_SPEC = {
    'num_vars': 53,
    'names': ['Farm___Crops___variables___Dryland_Winter_Barley___root_depth_m',
        'Farm___Crops___variables___Dryland_Winter_Barley___water_use_ML_per_Ha',
        'Farm___Crops___variables___Dryland_Winter_Barley___yield_per_Ha',
        'Farm___Crops___variables___Dryland_Winter_Canola___root_depth_m',
        'Farm___Crops___variables___Dryland_Winter_Canola___water_use_ML_per_Ha',
        'Farm___Crops___variables___Dryland_Winter_Canola___yield_per_Ha',
        'Farm___Crops___variables___Dryland_Winter_Wheat___root_depth_m',
        'Farm___Crops___variables___Dryland_Winter_Wheat___water_use_ML_per_Ha',
        'Farm___Crops___variables___Dryland_Winter_Wheat___yield_per_Ha',
        'Farm___Crops___variables___Irrigated_Winter_Barley___root_depth_m',
        'Farm___Crops___variables___Irrigated_Winter_Barley___water_use_ML_per_Ha',
        'Farm___Crops___variables___Irrigated_Winter_Barley___yield_per_Ha',
        'Farm___Crops___variables___Irrigated_Winter_Canola___root_depth_m',
        'Farm___Crops___variables___Irrigated_Winter_Canola___water_use_ML_per_Ha',
        'Farm___Crops___variables___Irrigated_Winter_Canola___yield_per_Ha',
        'Farm___Crops___variables___Irrigated_Winter_Wheat___root_depth_m',
        'Farm___Crops___variables___Irrigated_Winter_Wheat___water_use_ML_per_Ha',
        'Farm___Crops___variables___Irrigated_Winter_Wheat___yield_per_Ha',
        'Farm___Fields___soil___zone_10___TAW_mm',
        'Farm___Fields___soil___zone_11___TAW_mm',
        'Farm___Fields___soil___zone_12___TAW_mm',
        'Farm___Fields___soil___zone_1___TAW_mm',
        'Farm___Fields___soil___zone_2___TAW_mm',
        'Farm___Fields___soil___zone_3___TAW_mm',
        'Farm___Fields___soil___zone_4___TAW_mm',
        'Farm___Fields___soil___zone_5___TAW_mm',
        'Farm___Fields___soil___zone_6___TAW_mm',
        'Farm___Fields___soil___zone_7___TAW_mm',
        'Farm___Fields___soil___zone_8___TAW_mm',
        'Farm___Fields___soil___zone_9___TAW_mm',
        'Farm___Irrigations___Gravity___cost_per_Ha',
        'Farm___Irrigations___Gravity___head_pressure',
        'Farm___Irrigations___Gravity___irrigation_efficiency',
        'Farm___Irrigations___Gravity___pumping_cost_per_ML',
        'Farm___Irrigations___PipeAndRiser___cost_per_Ha',
        'Farm___Irrigations___PipeAndRiser___head_pressure',
        'Farm___Irrigations___PipeAndRiser___irrigation_efficiency',
        'Farm___Irrigations___PipeAndRiser___pumping_cost_per_ML',
        'Farm___Irrigations___Spray___cost_per_Ha',
        'Farm___Irrigations___Spray___head_pressure',
        'Farm___Irrigations___Spray___irrigation_efficiency',
        'Farm___Irrigations___Spray___pumping_cost_per_ML',
        'Farm___zone_10___Irrigation', 'Farm___zone_11___Irrigation',
        'Farm___zone_2___Irrigation', 'Farm___zone_4___Irrigation',
        'Farm___zone_6___Irrigation', 'Farm___zone_7___Irrigation',
        'Farm___zone_8___Irrigation', 'Farm___zone_9___Irrigation',
        'policy___goulburn_allocation_scenario', 'policy___gw_cap',
        'policy___gw_restriction'],
    'bounds': [(0.80008164104, 1.49988829764),
    (1.50055050742, 2.99888102069),
    (1.5019032420200003, 3.4997506932099998),
    (0.800586478968, 1.4996985073),
    (2.50048002895, 5.9984797603299995),
    (0.801052350325, 2.59824297051),
    (0.800504246618, 1.49975544648),
    (2.5014981435299997, 5.9979681912),
    (1.5004709810799999, 5.99716646463),
    (0.800280272497, 1.49937425734),
    (1.5009590614, 2.9992559947000004),
    (2.50329796931, 6.996816011819999),
    (0.800211596215, 1.49974890273),
    (2.0025975557, 5.99742468979),
    (1.3008100600299999, 4.99958661017),
    (0.8000586077680001, 1.7993585851400002),
    (2.50005748529, 5.99920182664),
    (1.5021921746899998, 7.99719295089),
    (150.013080285, 199.99630294),
    (145.01266211, 184.97447762599998),
    (145.036691741, 184.96132256099997),
    (145.017973816, 184.964659778),
    (145.009985077, 184.987775366),
    (100.017759932, 159.950281059),
    (100.00893349, 159.939807798),
    (150.002663759, 199.995911171),
    (150.049539279, 199.966206716),
    (75.011883698, 109.982509833),
    (100.007801344, 159.986958043),
    (145.015806747, 184.983072651),
    (2000.04766978, 2499.9660698000002),
    (8.00489093285, 14.999582054100001),
    (0.500092622216, 0.8998440697460001),
    (8.0072724319, 14.9995752798),
    (2000.65212205, 3299.41488388),
    (8.00365090987, 14.9983740134),
    (0.600018657025, 0.899703908987),
    (8.005434387660001, 14.9933485659),
    (2500.62094903, 3499.76177012),
    (25.0039236705, 34.9957834096),
    (0.7001056060199999, 0.8998137827079999),
    (30.000316497100002, 59.9914045149),
    (0.0, 1.0),
    (0.0, 1.0),
    (0.0, 1.0),
    (0.0, 2.0),
    (0.0, 1.0),
    (0.0, 1.0),
    (0.0, 1.0),
    (0.0, 2.0),
    (0.0, 2.0),
    (0.600156362739, 0.999676343195),
    (0.0, 1.0)]
}

In [ ]:
### Define problem spec for gravity irrigation
GRAV_SPEC = {
    'num_vars': 1,
    'names': [
        'Farm___Irrigations___Gravity___irrigation_efficiency'
    ],
    'bounds': [
        (0.50, 0.9),
    ]
}

In [ ]:
def to_numeric_samples(df):
    cat_cols = df.select_dtypes('object').columns
    numeric_df = df.copy()
    for col in numeric_df:
        if col in cat_cols:
            df[col] = df[col].astype('category')
            df[col] = df[col].cat.codes
        # End if
    # End for
    
    return df
# End to_numeric_samples()

In [ ]:
def plot_incremental_results(res, idx, name='Index', **kwargs):
    """kwargs: additional plot options to pass"""
    ax = pd.DataFrame({name: res}, index=idx).plot(kind='bar', **kwargs).legend(
                                       bbox_to_anchor=(1.05, 0.75)
                                    )
    return ax
# End plot_incremental_results()

In [ ]:
DATA_DIR = "../data/"
FIG_DIR = "../figures/"

In [ ]:
# Prep full results
all_inputs = pd.read_csv(f"{DATA_DIR}all_scenario_inputs.csv", index_col=0)
all_outputs = pd.read_csv(f"{DATA_DIR}all_scenario_outputs.csv", index_col=0)
all_outputs['Avg. $/ML'].fillna(all_outputs["Avg. Annual Profit ($M)"], inplace=True)

In [ ]:
tgt_param = "Farm___Irrigations___Gravity___irrigation_efficiency"
tgt_metric = "Recreation Index"  #  "SW Allocation Index"
template_df = pd.read_csv(f'{DATA_DIR}example_sample.csv', index_col=0)
is_perturbed = (template_df != template_df.iloc[0]).any()
perturbed_cols = template_df.loc[:, is_perturbed].columns